In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

from __future__ import division
import operator
import functools
import numpy as np
import scipy as sp
import pandas as pd
import math
import random
import pylab as pl
import matplotlib.pyplot as plt
import sklearn
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from collections import defaultdict
from math import log
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import mean_squared_error
import base64
import csv
import gzip
import zlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

from collections import namedtuple

from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.linear_model import SGDClassifier

Autosaving every 15 seconds


In [2]:
TRACE_NUM = 1000
import logging
from imp import reload
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

In [3]:
def to_utf8(text):
    if isinstance(text, str): text = text.encode('utf8', "ignore")
    return text

def convert2unicode(f):
    def tmp(text):
        if not isinstance(text, str): text = text.decode('utf8', "ignore")
        return f(text)
    return tmp

def convert2lower(f):
    def tmp(text):        
        return f(text.lower())
    return tmp

#P.S. Декораторы могут усложнять отладку, так что от них вполне можно отказаться и воспользоваться copy-paste

In [4]:
from html.parser import HTMLParser
import re

class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._text = []
        self._title = []
        self._in_title = False        

    def handle_data(self, data): 
        text = data.strip()        
        if len(text) > 0:
            text = re.sub('[ \t\r\n]+', ' ', text)
            if (self._in_title == True):
                self._title.append(text + ' ')
                self._in_title = False
            else:
                self._text.append(text + ' ')                

    def handle_starttag(self, tag, attrs):        
        if tag == 'p':
            self._text.append('\n\n')
        elif tag == 'br':
            self._text.append('\n')
        elif tag == 'title':
            self._in_title = True        

    def handle_startendtag(self, tag, attrs):        
        if tag == 'br':
            self._text.append('\n\n')
        elif tag == 'title':
            self._in_title = False        

    def text(self):
        return (''.join(self._title).strip(), ''.join(self._text).strip())

@convert2unicode
def html2text_parser(text):
    parser = TextHTMLParser()
    parser.feed(text)
    return parser.text()

In [5]:
@convert2lower
@convert2unicode
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    import pymorphy2
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

@convert2lower
@convert2unicode
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]

In [6]:
# return [title, text, text in <a>]
def html2word(raw_html, to_text=html2text_parser, tokenizer=easy_tokenizer):
    title, data = to_text(raw_html)
    return (list(tokenizer(title.lower())), list(tokenizer(data.lower())))

In [7]:
DocItem = namedtuple('DocItem', ['doc_id', 'is_spam', 'url', 'features'])

def load_csv(input_file_name):    
    train = []
    ans = []
    with gzip.open(input_file_name) if input_file_name.endswith('gz') else open(input_file_name) as input_file:            
        headers = input_file.readline()        
        for i, line in enumerate(input_file):            
            trace(i)
            parts = line.decode('utf-8').strip().split('\t')
            url_id = int(parts[0])
            mark = int(parts[1])
            url = parts[2]
            pageInb64 = parts[3]
            html_data = base64.b64decode(pageInb64)            
            title, data = html2word(html_data)
            train.append(' '.join(data))
            ans.append(mark)
    return (train, ans)

In [27]:
TRAIN_DATA_FILE  = 'kaggle_train_data_tab.csv.gz'
train, ans = load_csv(TRAIN_DATA_FILE)

23:01:06 INFO:Complete items 00000
23:01:26 INFO:Complete items 01000
23:01:44 INFO:Complete items 02000
23:02:02 INFO:Complete items 03000
23:02:18 INFO:Complete items 04000
23:02:36 INFO:Complete items 05000
23:02:53 INFO:Complete items 06000
23:03:14 INFO:Complete items 07000


In [29]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', SGDClassifier())])

text_clf.fit(train, ans)
predicted = text_clf.predict(train)
print('f1 score = ', f1_score(ans, predicted))


f1 score =  0.9950799508


In [32]:
# from sklearn.neighbors import KNeighborsClassifier

text_clf2 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])

text_clf2.fit(train, ans)
print('fit ok')
predicted = text_clf2.predict(train)
print('predict ok')
print('f1 score = ', f1_score(ans, predicted))


fit ok
predict ok
f1 score =  0.979762061548


In [33]:
text_clf3 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', RandomForestClassifier())])

text_clf3.fit(train, ans)
print('fit ok')
predicted = text_clf3.predict(train)
print('predict ok')
print('f1 score = ', f1_score(ans, predicted))



fit ok
predict ok
f1 score =  0.999034579603


In [28]:
print(len(train))
for x in train2:
    train.append(' '.join(x))
print(len(train))
ans = ans + ans2
# train2 = []
# ans2 = []


7044
20705


In [35]:
TEST_DATA_FILE  = 'kaggle_test_data_tab.csv.gz'

train2 = []
ans2 = []
fout2 = open('my_submission2.csv' , 'w')
fout3 = open('my_submission3.csv' , 'w')
with open('my_submission.csv' , 'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Id','Prediction'])
    writer2 = csv.writer(fout2)
    writer2.writerow(['Id','Prediction'])

    writer3 = csv.writer(fout3)
    writer3.writerow(['Id','Prediction'])

    with gzip.open(TEST_DATA_FILE) if TEST_DATA_FILE.endswith('gz') else open(TEST_DATA_FILE) as input_file:            
        headers = input_file.readline()        
        for i, line in enumerate(input_file):
            trace(i)
            parts = line.decode('utf-8').strip().split('\t')
            url_id = int(parts[0])        
            title, data = html2word(base64.b64decode(parts[3]))   
            ss = [' '.join(data)]
            predicted = [text_clf.predict(ss)[0], text_clf2.predict(ss)[0], text_clf3.predict(ss)[0]]
            
            writer2.writerow([url_id, predicted[1]])
            writer3.writerow([url_id, predicted[2]])
            predicted.sort()
            writer.writerow([url_id, predicted[1]])
            cnt = sum(predicted)
            if cnt == 0:
                train2.append(ss)
                ans2.append(0)
            elif cnt == 3:
                train2.append(ss)
                ans2.append(1)                        
fout2.close()
fout3.close()

23:31:55 INFO:Complete items 00000
23:34:10 INFO:Complete items 01000
23:36:49 INFO:Complete items 02000
23:38:41 INFO:Complete items 03000
23:40:32 INFO:Complete items 04000
23:42:24 INFO:Complete items 05000
23:44:19 INFO:Complete items 06000
23:46:09 INFO:Complete items 07000
23:48:03 INFO:Complete items 08000
23:49:54 INFO:Complete items 09000
23:51:55 INFO:Complete items 10000
23:53:44 INFO:Complete items 11000
23:55:33 INFO:Complete items 12000
23:57:21 INFO:Complete items 13000
23:59:13 INFO:Complete items 14000
00:01:15 INFO:Complete items 15000
00:03:19 INFO:Complete items 16000
